In [ ]:
import sys
sys.path.append("../")
import utils

import metrics_wrapper
from budget_allocation import select_subset

In [ ]:
# METRIC = "comet20"
METRIC = "chrf"
PATTERNS_REF = ["R1", "R2", "R4", "R3"]
metric_score = {
    pattern_ref:utils.load_metric_scores(
            f"../../computed/metric_scores_{METRIC}.json",
            pattern_ref=pattern_ref,
            aggregate="average",
            refids_path="../../computed/metric_scores_none.json"
        )
    for pattern_ref in PATTERNS_REF
}

In [ ]:
# correlations figure

import numpy as np
import collections
import tqdm

data_wmt = utils.load_wmt(annotation_path="../../data/annotations.json", wmt_path="../../data/data_tmp/")

S = set(x["src"] for x in data_wmt)

def evaluate_subset(R):
    # maps src -> reference vendor
    R_map = collections.defaultdict(list)
    for i, values in R.items():
        for x in values:
            R_map[x].append(i)
    R_map = {
        x: list(values)
        for x, values in R_map.items()
    }

    data_wmt_local = [
        {
            "src": x["src"],
            "tgt": x["tgt"],
            "system": x["system"],
            "human": x["score"],
            "metric": np.average([
                metric_score[f"R{i}"][(x["src"], x["tgt"])]
                for i in R_map[x["src"]]
            ])
        }
        for x in data_wmt
    ]
    corr, _ = utils.compute_segment_tau(data_wmt_local)
    return corr

Bs = [160, 160*2, 160*3, 160*4, 160*5, 160*6]
QUALITY_LAMBDAs = np.linspace(0.001, 0.7, 10)
COUNTs = {}
for quality_lambda in tqdm.tqdm(QUALITY_LAMBDAs):
    for B in Bs:
        corrs = []
        for _ in range(50):
            R = select_subset(
                S=S,
                B=B,
                quality_lambda=quality_lambda,
                temp=0.5
            )
            corr = evaluate_subset(R)
            corrs.append(corr)
        COUNTs[(B, quality_lambda)] = np.average(corrs)

In [ ]:
# correlations figure

import matplotlib.pyplot as plt

img = np.zeros((len(Bs), len(QUALITY_LAMBDAs)))

for b_i, b_v in enumerate(Bs):
    for l_i, l_v in enumerate(QUALITY_LAMBDAs):
        img[b_i, l_i] = COUNTs[(b_v, l_v)]

img = img.T

plt.figure(figsize=(1.9, 2))
for row_i, col in enumerate(img.T):
    if row_i == 0:
        continue
    col_i = np.argmax(col)
    plt.scatter(
        x=[row_i], y=[col_i],
        marker="*", s=50,
        color="black"
    )


plt.imshow(
    img,
    cmap="gray",
    aspect="auto",
    vmin=0.07 if METRIC == "chrf" else 0.165,
    vmax=0.15 if METRIC == "chrf" else 0.192,
)
a = plt.colorbar(
    aspect=10, pad=0.03,
    ticks=
        [0.08, 0.11, 0.14] if METRIC == "chrf" else
        None
)
a.outline.set_linewidth(0)



ax = plt.gca()
plt.title(metrics_wrapper.METRIC_NAMES[METRIC], fontsize=12)
plt.xlabel("Budget", labelpad=-5)
plt.xticks(
    list(range(len(Bs)))[::5],
    [f"${b//160}|S|$" for b in Bs][::5]
)
plt.yticks(
    [1, len(QUALITY_LAMBDAs)//2-0.5, len(QUALITY_LAMBDAs)-1-1],
    ["$\\rightarrow$Quantity", "$\\lambda$", "Quality$\\leftarrow$"],
    rotation=90,
    va="center",
    fontsize=9,
)

ax.tick_params(axis='y', which='major', pad=-1)
ax.tick_params(left=False, bottom=True)
ax.spines[['left', 'right', 'top', 'bottom']].set_visible(False)
plt.tight_layout(pad=0)

plt.savefig(f"../../computed/budget_allocation_{METRIC}.pdf")

In [ ]:
# reference count figure

import numpy as np
import collections
import tqdm

data_wmt = utils.load_wmt(annotation_path="../../data/annotations.json", wmt_path="../../data/data_tmp/")

S = set(x["src"] for x in data_wmt)

def evaluate_subset(R):
    # maps src -> reference vendor
    R_map = collections.defaultdict(list)
    for i, values in R.items():
        for x in values:
            R_map[x].append(i)
    R_map = {
        x: len(values)
        for x, values in R_map.items()
    }
    avg_per_seg = np.average(list(R_map.values()))
    return avg_per_seg

Bs = [160, 160*2, 160*3, 160*4, 160*5, 160*6]
QUALITY_LAMBDAs = np.linspace(0.001, 0.7, 10)
COUNTs = {}
for quality_lambda in tqdm.tqdm(QUALITY_LAMBDAs):
    for B in Bs:
        counts = []
        for _ in range(10):
            R = select_subset(
                S=S,
                B=B,
                quality_lambda=quality_lambda,
                temp=0.5
            )
            count = evaluate_subset(R)
            counts.append(count)
        COUNTs[(B, quality_lambda)] = np.average(counts)

In [ ]:
# reference count figure

import matplotlib.pyplot as plt

img = np.zeros((len(Bs), len(QUALITY_LAMBDAs)))

for b_i, b_v in enumerate(Bs):
    for l_i, l_v in enumerate(QUALITY_LAMBDAs):
        img[b_i, l_i] = COUNTs[(b_v, l_v)]
img = img.T

plt.figure(figsize=(4, 2))

for row_i, col in enumerate(img):
    for col_i, value in enumerate(col):
        plt.text(
            x=col_i, y=row_i,
            s=f"{value:.1f}",
            ha="center", va="center",
            color="white" if value < 2.1 else "black",
            fontsize=8,
        )

plt.imshow(
    img,
    cmap="gray",
    aspect="auto",
    vmin=0.8, vmax=4,
)
a = plt.colorbar(
    aspect=10, pad=0.03,
    ticks=None
)
a.outline.set_linewidth(0)

ax = plt.gca()
plt.title("Number of references per segment", fontsize=12)
plt.xlabel("Budget", labelpad=-10)
plt.xticks(
    list(range(len(Bs)))[::5],
    [f"${b//160}|S|$" for b in Bs][::5]
)
plt.yticks(
    [1, len(QUALITY_LAMBDAs)//2-0.5, len(QUALITY_LAMBDAs)-1-1],
    ["$\\rightarrow$Quantity", "$\\lambda$", "Quality$\\leftarrow$"],
    rotation=90,
    va="center",
    fontsize=9,
)

ax.tick_params(axis='y', which='major', pad=-1)
ax.tick_params(left=False, bottom=True)
ax.spines[['left', 'right', 'top', 'bottom']].set_visible(False)
plt.tight_layout(pad=0)

plt.savefig(f"../../computed/budget_avgref.pdf")